<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Изучение-данных" data-toc-modified-id="Изучение-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Изучение данных</a></span></li><li><span><a href="#Предобработка-данных" data-toc-modified-id="Предобработка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Предобработка данных</a></span><ul class="toc-item"><li><span><a href="#Пропуски,-дубли,-адекватость-данных,-переименование-столбцов" data-toc-modified-id="Пропуски,-дубли,-адекватость-данных,-переименование-столбцов-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Пропуски, дубли, адекватость данных, переименование столбцов</a></span></li><li><span><a href="#Подготовка-признаков" data-toc-modified-id="Подготовка-признаков-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Подготовка признаков</a></span></li></ul></li><li><span><a href="#Обучение-без-учёта-дисбаланса-классов" data-toc-modified-id="Обучение-без-учёта-дисбаланса-классов-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Обучение без учёта дисбаланса классов</a></span><ul class="toc-item"><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Случайный лес</a></span></li><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li></ul></li><li><span><a href="#Обучение-с-учётом-дисбаланса" data-toc-modified-id="Обучение-с-учётом-дисбаланса-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Обучение с учётом дисбаланса</a></span><ul class="toc-item"><li><span><a href="#class_weight='balanced'" data-toc-modified-id="class_weight='balanced'-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span><code>class_weight='balanced'</code></a></span><ul class="toc-item"><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-4.1.1"><span class="toc-item-num">4.1.1&nbsp;&nbsp;</span>Случайный лес</a></span></li><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-4.1.2"><span class="toc-item-num">4.1.2&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li></ul></li></ul></li></ul></div>

# Отток клиентов

**Цель**: спрогнозировать, уйдёт клиент из банка в ближайшее время или нет.

У нас имеются исторические данные о поведении клиентов и расторжении договоров с банком. Построим модель с предельно большим значением F1-меры (нужно довести метрику до 0.59). Дополнительно измерим AUC-ROC и сравним её значение с F1-мерой.

## Изучение данных

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sns
import os
#from catboost import CatBoostClassifier, Pool
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
#from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
#from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
#from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
#import warnings
#from random import seed
#from random import random
#import statistics

In [2]:
pth1 = '/datasets/Churn.csv'
pth2 = 'Churn.csv'
if os.path.exists(pth1):
    df = pd.read_csv(pth1)
elif os.path.exists(pth2):
    df = pd.read_csv(pth2)
else:
    print('Something is wrong')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [4]:
df.head(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


В таблице 14 столбцов, 10000 строк. Пропуски есть только в столбце `tenure`.

Согласно документации:

`RowNumber` — индекс строки в данных;
<br>`CustomerId` — уникальный идентификатор клиента;
<br>`Surname` — фамилия;
<br>`CreditScore` — кредитный рейтинг;
<br>`Geography` — страна проживания;
<br>`Gender` — пол;
<br>`Age` — возраст;
<br>`Tenure` — сколько лет человек является клиентом банка;
<br>`Balance` — баланс на счёте;
<br>`NumOfProducts` — количество продуктов банка, используемых клиентом;
<br>`HasCrCard` — наличие кредитной карты;
<br>`IsActiveMember` — активность клиента;
<br>`EstimatedSalary` — предполагаемая зарплата;

`Exited` — факт ухода клиента (целевой признак).

## Предобработка данных

### Пропуски, дубли, адекватость данных, переименование столбцов

Столбцы именованы в стиле `CamelCase`, приведём к `snake_case`.

In [5]:
df.columns = [ 'row_number', 'customer_id', 'surname', 'credit_score', 'geography', 'gender', 'age', 'tenure', 
              'balance', 'num_of_products', 'has_credit_card', 'is_active_member', 'estimated_salary', 'exited' ]
df.columns

Index(['row_number', 'customer_id', 'surname', 'credit_score', 'geography',
       'gender', 'age', 'tenure', 'balance', 'num_of_products',
       'has_credit_card', 'is_active_member', 'estimated_salary', 'exited'],
      dtype='object')

Проверим пропуски в `tenure`.

In [6]:
df['tenure'].value_counts(dropna=False)

1.0     952
2.0     950
8.0     933
3.0     928
5.0     927
7.0     925
NaN     909
4.0     885
9.0     882
6.0     881
10.0    446
0.0     382
Name: tenure, dtype: int64

Это почти 10% данных, но в целях обучения моделей, думаю, лучше будет удалить пропуски.

In [7]:
df = df.dropna(subset=['tenure'])

Проверим дубли.

In [8]:
df.duplicated().sum()

0

Дублей нет.

Столбцы `row_number`, `customer_id`, `surname` нельзя/нецелесообразно использовать для обучения модели, удалим их.

In [9]:
df = df.drop(columns=[  'row_number', 'customer_id', 'surname'])

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9091 entries, 0 to 9998
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   credit_score      9091 non-null   int64  
 1   geography         9091 non-null   object 
 2   gender            9091 non-null   object 
 3   age               9091 non-null   int64  
 4   tenure            9091 non-null   float64
 5   balance           9091 non-null   float64
 6   num_of_products   9091 non-null   int64  
 7   has_credit_card   9091 non-null   int64  
 8   is_active_member  9091 non-null   int64  
 9   estimated_salary  9091 non-null   float64
 10  exited            9091 non-null   int64  
dtypes: float64(3), int64(6), object(2)
memory usage: 852.3+ KB


Остаётся 10 признаков и 1 целевой признак.

Проверим данные в остальных столбцах.

In [11]:
df.credit_score.describe()

count    9091.000000
mean      650.736553
std        96.410471
min       350.000000
25%       584.000000
50%       652.000000
75%       717.000000
max       850.000000
Name: credit_score, dtype: float64

In [12]:
df.geography.value_counts()

France     4550
Germany    2293
Spain      2248
Name: geography, dtype: int64

In [13]:
df.gender.value_counts()

Male      4974
Female    4117
Name: gender, dtype: int64

In [14]:
df.age.describe()

count    9091.000000
mean       38.949181
std        10.555581
min        18.000000
25%        32.000000
50%        37.000000
75%        44.000000
max        92.000000
Name: age, dtype: float64

In [15]:
df.balance.describe()

count      9091.000000
mean      76522.740015
std       62329.528576
min           0.000000
25%           0.000000
50%       97318.250000
75%      127561.890000
max      250898.090000
Name: balance, dtype: float64

In [16]:
df.num_of_products.value_counts()

1    4617
2    4184
3     234
4      56
Name: num_of_products, dtype: int64

In [17]:
df.has_credit_card.value_counts()

1    6409
0    2682
Name: has_credit_card, dtype: int64

In [18]:
df.is_active_member.value_counts()

1    4687
0    4404
Name: is_active_member, dtype: int64

In [19]:
df.estimated_salary.describe()

count      9091.000000
mean     100181.214924
std       57624.755647
min          11.580000
25%       51227.745000
50%      100240.200000
75%      149567.210000
max      199992.480000
Name: estimated_salary, dtype: float64

In [20]:
df.exited.value_counts()

0    7237
1    1854
Name: exited, dtype: int64

### Подготовка признаков

2 признака: `geography`, `gender` - нужно перевести в числовой вид. Воспользуемся техникой *One-Hot Encoding*, т.к. она работает со всеми моделями, и разных значений в этих признаках не так много, поэтому не будет слишком много дополнительных столбцов.

In [21]:
df_ohe = pd.get_dummies(df, drop_first=True)

Масштабирум количественные признаки, чтобы они были равнозначны для моделей.

In [22]:
numeric = [ 'credit_score', 'age', 'tenure', 'balance', 'num_of_products', 'estimated_salary' ]

scaler = StandardScaler()
scaler.fit(df_ohe[numeric])
df_ohe[numeric] = scaler.transform(df_ohe[numeric])

In [23]:
df_ohe.head(2)

,credit_score,age,tenure,balance,num_of_products,has_credit_card,is_active_member,estimated_salary,exited,geography_Germany,geography_Spain,gender_Male
0,-0.329200,0.289040,-1.035627,-1.227780,-0.912601,1,1,0.020264,1,0,0,0
1,-0.443301,0.194298,-1.381103,0.116887,-0.912601,0,1,0.214527,0,0,1,0


## Обучение без учёта дисбаланса классов

Исследуем баланс классов, обучим модель без учёта дисбаланса. Кратко опишем выводы.

In [24]:
df.exited.value_counts() / df.exited.count()

0    0.796062
1    0.203938
Name: exited, dtype: float64

Есть дисбаланс целевых классов: 20 / 80.

Разделим выборку на признаки, целевой признак, а также на обучающую, валидационную и тестовую выборки.

In [43]:
target = df_ohe['exited']
features = df_ohe.drop('exited', axis=1)

features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, 
    test_size=0.3, shuffle=True, stratify=target, random_state=246758)

features_test, features_valid, target_test, target_valid = train_test_split(
    features_valid, target_valid, 
    test_size=0.5, shuffle=True, stratify=target_valid, random_state=9421023)

print(features_train.shape) 
print(features_valid.shape) 
print(features_test.shape) 
print('\n', target_train.mean(), target_valid.mean(), target_test.mean(), sep='\t')

(6363, 11)
(1364, 11)
(1364, 11)

	0.203991827754204	0.20381231671554254	0.20381231671554254


Подготовим также увеличенные тренировочные выборки, которые можно использовать для обучения после подбора гиперпараметров либо сразу в GridSearch.

In [44]:
features_train_2 = pd.concat([features_train, features_valid], ignore_index=True)
target_train_2 = pd.concat([target_train, target_valid], ignore_index=True)
print(features_train_2.shape, target_train_2.shape)

(7727, 11) (7727,)


### Случайный лес

In [45]:
%%time
model = RandomForestClassifier(random_state=12345)

params = dict(n_estimators=range(100,136,3), max_depth=range(13,20,1))
grid = GridSearchCV(model, params, n_jobs=4, verbose=1, scoring='f1', cv=2)
grid.fit(features_train, target_train);

Fitting 2 folds for each of 84 candidates, totalling 168 fits
CPU times: total: 1.27 s
Wall time: 25.8 s


GridSearchCV(cv=2, estimator=RandomForestClassifier(random_state=12345),
             n_jobs=4,
             param_grid={'max_depth': range(13, 20),
                         'n_estimators': range(100, 136, 3)},
             scoring='f1', verbose=1)

In [46]:
print(grid.best_params_)
best_forest = grid.best_estimator_
print(best_forest)
print('f1 лучшей модели на обучающей выборке:', grid.best_score_)
predicted_valid = best_forest.predict(features_valid)
print('f1 лучшей модели на валидационной выборке:', f1_score(target_valid, predicted_valid))

{'max_depth': 15, 'n_estimators': 103}
RandomForestClassifier(max_depth=15, n_estimators=103, random_state=12345)
f1 лучшей модели на обучающей выборке: 0.5637985297690686
f1 лучшей модели на валидационной выборке: 0.5637583892617449


Полученное без балансировки классов значение ***f1*** даже при подборе гиперпараметров - не слишком велико. Пока не удалось преодолеть установленный порог **0.59**.

### Логистическая регрессия

In [52]:
model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
print("F1:", f1_score(target_valid, predicted_valid))

F1: 0.3298429319371728


Очень низкий показатель.

## Обучение с учётом дисбаланса
Теперь будем учитывать дисбаланс классов. Обучим разные модели и выберем лучшую.

### `class_weight='balanced'`

#### Случайный лес

In [49]:
%%time
model = RandomForestClassifier(max_depth=15, n_estimators=103, random_state=12345,
                              class_weight='balanced')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
print('f1:', f1_score(target_valid, predicted_valid))

f1: 0.5897435897435898
CPU times: total: 719 ms
Wall time: 704 ms


Пока не удалось получить существенный выигрыш по сравнению с предыдущим результатом случайного леса.

#### Логистическая регрессия

In [53]:
model = LogisticRegression(random_state=12345, solver='liblinear', class_weight='balanced')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
print("F1:", f1_score(target_valid, predicted_valid))

F1: 0.48258706467661694


Уже существенный выигрыш по сравнению с результатом лог.регрессии без указания параметра `class_weight`, но всё ещё очень далеко до 0.59.